In [1]:
import pandas as pd
import numpy as np

# Data Check
- Session 테이블의 TOT_PAG_VIEW_CT와 TOT_SESS_HR_V가 Nan인 경우 비정상적인 접근(ex. 접속하자마자 종료된 경우)이라고 생각하여 삭제하였음

In [2]:
prod = pd.read_csv("01.Pruduct.csv")
sear = pd.read_csv("02.Search1.csv")
sess = pd.read_csv("05.Session.csv")
mast = pd.read_csv("06.Master.csv")
print(prod.shape)
print(sear.shape)
print(sess.shape)
print(mast.shape)

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(5024906, 8)
(2884943, 4)
(2712907, 9)
(847652, 5)


In [9]:
print(prod.isnull().sum())
print(sear.isnull().sum())
print(sess.isnull().sum())
print(mast.isnull().sum())

CLNT_ID      0
SESS_ID      0
HITS_SEQ     0
PD_C         0
PD_ADD_NM    0
PD_BRA_NM    0
PD_BUY_AM    0
PD_BUY_CT    0
dtype: int64
CLNT_ID       0
SESS_ID       0
KWD_NM        0
SEARCH_CNT    0
dtype: int64
CLNT_ID                0
SESS_ID                0
SESS_SEQ               0
SESS_DT                0
TOT_PAG_VIEW_CT      274
TOT_SESS_HR_V      14202
DVC_CTG_NM             0
ZON_NM                 0
CITY_NM                0
dtype: int64
PD_C        0
PD_NM       0
CLAC1_NM    0
CLAC2_NM    0
CLAC3_NM    0
dtype: int64


In [10]:
sess[sess['TOT_SESS_HR_V'].isnull()].head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
38,5878756,7465325,5,20180529,1.0,NaN,mobile,Chungcheongnam-do,Cheonan-si
64,6065738,9564520,26,20180424,1.0,NaN,mobile,Gyeonggi-do,Gwangmyeong-si
65,6065738,9564529,25,20180424,1.0,NaN,mobile,Gyeonggi-do,Gwangmyeong-si
246,6003295,1035552,2,20180913,1.0,NaN,desktop,Daejeon,Daejeon
604,5946142,9600252,4,20180423,1.0,NaN,mobile,Seoul,Seoul


In [11]:
sess[sess['TOT_PAG_VIEW_CT'].isnull()].head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
11409,5075856,6836578,19,20180608,NaN,NaN,mobile,Seoul,Seoul
26628,5368135,4391880,10,20180718,NaN,NaN,mobile,Chungcheongnam-do,Yeongi-gun
43241,5687186,852560,40,20180916,NaN,NaN,mobile,Seoul,Seoul
58494,6104032,1568619,78,20180904,NaN,NaN,mobile,Chungcheongnam-do,Cheonan-si
64700,3735745,6884635,1,20180607,NaN,NaN,desktop,Gwangju,Gwangju


In [3]:
sess = sess.dropna()
print(sess.isnull().sum())
print(sess.shape)

CLNT_ID            0
SESS_ID            0
SESS_SEQ           0
SESS_DT            0
TOT_PAG_VIEW_CT    0
TOT_SESS_HR_V      0
DVC_CTG_NM         0
ZON_NM             0
CITY_NM            0
dtype: int64
(2698696, 9)


In [4]:
sess['TOT_SESS_HR_V'] = sess['TOT_SESS_HR_V'].map(lambda x: x.replace(",", ""))
sess.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,1890,mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,1604,mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,1035,mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298,mobile,Seoul,Seoul


# Clustering
- 세션 일련 번호, 총 페이지 조회 건수, 총 세션 시간 값, 기기 유형, 행동을 기준으로 클러스터링

In [5]:
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

In [6]:
act_buy = prod[['SESS_ID']]
act_buy['ACTION'] = 'buy'

act_sear = sear[['SESS_ID']]
act_sear['ACTION'] = 'search'

act = pd.concat([act_buy, act_sear], axis=0, sort=False)
print(act['ACTION'].unique())
act = act.drop_duplicates()
act = act.groupby('SESS_ID').agg(lambda x: x.tolist())

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


['buy' 'search']


In [7]:
#act['ACTION'].unique

act['ACTION'] = act['ACTION'].map(lambda x: "".join(x) if len(x)<2 else ",".join(x))
print(act['ACTION'].unique())         #.unique 는 유일한 값 찾는것.. 

['buy' 'buy,search']


In [8]:
print(act.tail())

              ACTION
SESS_ID             
10977908         buy
10977924         buy
10977932  buy,search
10977936  buy,search
10977941         buy


In [9]:
df = pd.merge(sess, act, left_on='SESS_ID', right_on='SESS_ID', how='left')
df.shape

(2698696, 10)

In [10]:
df = df[['CLNT_ID', 'SESS_ID','SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']]
df.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,mobile,"buy,search"
1,5873599,6616320,21,105.0,1604,mobile,"buy,search"
2,5873599,5886172,40,41.0,632,mobile,"buy,search"
3,5873884,1050889,15,160.0,1035,mobile,"buy,search"
4,5874461,10298270,5,13.0,298,mobile,buy


## ACTION별로 클러스터링하기 

In [11]:
from sklearn.preprocessing import LabelEncoder

encode_DVC = LabelEncoder()
encode_DVC.fit(df['DVC_CTG_NM'])

df['DVC_CTG_NM'] = encode_DVC.transform(df['DVC_CTG_NM'])

encode_ACT = LabelEncoder()
df.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,1,"buy,search"
1,5873599,6616320,21,105.0,1604,1,"buy,search"
2,5873599,5886172,40,41.0,632,1,"buy,search"
3,5873884,1050889,15,160.0,1035,1,"buy,search"
4,5874461,10298270,5,13.0,298,1,buy


In [13]:
act = df.iloc[:,2:].groupby('ACTION').agg(['size','sum'])
act[:5]
#data = df.iloc[:,3:].groupby('CLUSTER').agg(['size','mean','sum'])

SESS_SEQ            TOT_PAG_VIEW_CT              TOT_SESS_HR_V  \
               size        sum            size          sum          size   
ACTION                                                                      
buy         1332423  212394228         1332423   68510581.0       1332423   
buy,search  1366273  270168235         1366273  162504052.0       1366273   

                                                              DVC_CTG_NM  \
                                                          sum       size   
ACTION                                                                     
buy         2984442632881598887561492839118133521904855602...    1332423   
buy,search  1890160463210353409763099317414378610383121104...    1366273   

                     
                sum  
ACTION               
buy         1137282  
buy,search  1164063

In [31]:
act2 = df.groupby('ACTION').agg({'SESS_SEQ':'size'})
act2

,SESS_SEQ
ACTION,
buy,1332423
"buy,search",1366273


### action과 session 방문횟수, 시간, 사용기기에 따라 클러스터?

In [17]:
from sklearn.cluster import MiniBatchKMeans, KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

In [ ]:
cols = ['ACTION']

In [ ]:
for num in range(3,7):
    print("For n_clusters = {}".format(num))
    clusterer = KMeans(n_clusters=num)
    preds = clusterer.fit_predict(df[cols])
    score = silhouette_score(df[cols], preds)
    print("silhouette score is {}".format(score))

In [21]:
cols = ['SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']
#for num in range(3,7):
#print("For n_clusters = {}".format(num))
clusterer = MiniBatchKMeans(n_clusters=5)
preds = clusterer.fit_predict(df[cols])
print(clusterer.score(df[cols]))

-803091239343.884


In [17]:
#score = silhouette_score(df[cols], preds)
#print("silhouette score is {}".format(score))

In [61]:
df['CLUSTER'] = preds
df.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,1,1,2
1,5873599,6616320,21,105.0,1604,1,1,4
2,5873599,5886172,40,41.0,632,1,1,0
3,5873884,1050889,15,160.0,1035,1,1,4
4,5874461,10298270,5,13.0,298,1,0,0


In [62]:
cluster = df['CLUSTER']
cluster.head()

0    2
1    4
2    0
3    4
4    0
Name: CLUSTER, dtype: int32

In [63]:
data = df.iloc[:,3:].groupby('CLUSTER').agg(['size','mean','sum'])
data[:5]

TOT_PAG_VIEW_CT                         DVC_CTG_NM                     \
                   size        mean         sum       size      mean      sum   
CLUSTER                                                                         
0               1226491   37.744698  46293532.0    1226491  0.871851  1069318   
1                230012  205.051649  47164340.0     230012  0.816897   187896   
2                459397  131.267126  60303724.0     459397  0.841055   386378   
3                 62732  281.248597  17643287.0      62732  0.728958    45729   
4                720064   82.783961  59609750.0     720064  0.849958   612024   

          ACTION                    
            size      mean     sum  
CLUSTER                             
0        1226491  0.342672  420284  
1         230012  0.785216  180609  
2         459397  0.675499  310322  
3          62732  0.850332   53343  
4         720064  0.557888  401715

In [19]:
data2 = df.groupby('CLUSTER').agg(lambda x: x.tolist())
data2.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
CLUSTER,,,,,,,
0,"[5873599, 5874461, 5874701, 5874829, 5874829, ...","[5886172, 10298270, 608577, 1666766, 1559438, ...","[40, 5, 65, 87, 92, 2, 16, 13, 5, 47, 156, 165...","[41.0, 13.0, 41.0, 36.0, 24.0, 4.0, 159.0, 33....","[632, 298, 444, 340, 317, 263, 786, 312, 666, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ..."
1,"[5874829, 5878622, 5878892, 5914916, 5917727, ...","[1694242, 352440, 1554809, 2188919, 612280, 20...","[85, 88, 122, 12, 181, 99, 208, 227, 159, 118,...","[154.0, 142.0, 56.0, 244.0, 142.0, 285.0, 101....","[3099, 3352, 2543, 3119, 3301, 3612, 2546, 238...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, ..."
2,"[5873599, 5873599, 5873884, 5874829, 5876333, ...","[8641867, 6616320, 1050889, 1670494, 456326, 2...","[9, 21, 15, 86, 15, 17, 366, 429, 362, 517, 36...","[82.0, 105.0, 160.0, 103.0, 160.0, 72.0, 53.0,...","[1890, 1604, 1035, 976, 1038, 1104, 1022, 2019...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, ..."
3,"[5878512, 5993798, 5993798, 5880431, 5880706, ...","[658549, 1052000, 738093, 8358105, 534772, 108...","[2, 42, 59, 4, 600, 128, 49, 52, 1, 3, 3, 6, 3...","[115.0, 422.0, 250.0, 207.0, 323.0, 361.0, 467...","[8172, 6399, 6321, 6698, 6399, 6831, 7241, 754...","[0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
4,"[5874900, 6075137, 6070142, 6072805, 6072961, ...","[10229983, 3237950, 1111137, 841910, 10312575,...","[7, 307, 114, 1, 236, 271, 81, 182, 28, 50, 58...","[207.0, 333.0, 52.0, 63.0, 267.0, 316.0, 221.0...","[4143, 4103, 4039, 6077, 4851, 3766, 5717, 483...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ..."


In [54]:
check = data.groupby('CLUSTER').agg(lambda x:x.tolist())
check.transpose()

CLUSTER                                   0                     1  \
TOT_PAG_VIEW_CT size              [1328516]              [425629]   
                mean    [39.87683249580735]  [151.97815938293678]   
                sum            [52977010.0]          [64686312.0]   
DVC_CTG_NM      size              [1328516]              [425629]   
                mean   [0.8703892162382688]  [0.8342876072823985]   
                sum               [1156326]              [355097]   
ACTION          size              [1328516]              [425629]   
                mean  [0.35479211390754795]  [0.7134100355004006]   
                sum                [471347]              [303648]   

CLUSTER                                  2                     3  \
TOT_PAG_VIEW_CT size               [28768]              [763073]   
                mean  [297.54348581757506]   [90.39180261914653]   
                sum            [8559731.0]          [68975544.0]   
DVC_CTG_NM      size               [28768]              [763073]   
                mean  [0.6754379866518354]  [0.8493866248707529]   
                sum                [19431]              [648144]   
ACTION          size               [28768]              [763073]   
                mean   [0.858071468298109]  [0.5793678979599592]   
                sum                [24685]              [442100]   

CLUSTER                                  4  
TOT_PAG_VIEW_CT size              [152710]  
                mean   [234.5362844607426]  
                sum           [35816036.0]  
DVC_CTG_NM      size              [152710]  
                mean  [0.8011721563748281]  
                sum               [122347]  
ACTION          size              [152710]  
                mean  [0.8152249361534936]  
                sum               [124493]

### 알 수 있는 점
-cluster0 이 가장 많은 데이터를 포함하고 있으며 cluster2가 가장 적은 데이터를 포함하고 있다. 
-가장 많은 데이터를 갖지만 페이지 방문시간은 작다..! 적은시간을 들여 에 구매하는 사람이 많다는 것을 알 수 있다
또한, 많은 데이터, 적은 action 평균을 갖는 다는 것은 충동구매(buy만)가 많다는 것!

=> 많은 사람들이 짧은 시간에 충동구매를 한다.!

- cluster2가 가장 적은 데이터를 갖지만 action의 평균이 가장 높은데 이는 검색해서 구매하는, 계획 구매가 많다는 것을 알 수 있다
적은 수의 사람이 계획구매를 한다. 

# Frequent Pattern Mining

In [55]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [64]:
data2 = df.groupby('CLUSTER').agg(lambda x: x.tolist())
data2.transpose()

CLUSTER,0,1,2,3,4
CLNT_ID,"[5873599, 5874461, 5874701, 5874829, 5874829, ...","[5874900, 5878622, 5917727, 5918023, 6075137, ...","[5873599, 5874829, 5876800, 5878756, 5878847, ...","[5878512, 6072805, 5987485, 5993798, 5993798, ...","[5873599, 5873884, 5874829, 5876333, 5876382, ..."
SESS_ID,"[5886172, 10298270, 608577, 1666766, 1559438, ...","[10229983, 352440, 612280, 2059825, 3237950, 1...","[8641867, 1694242, 7153099, 7517904, 153865, 1...","[658549, 841910, 1853495, 1052000, 829396, 738...","[6616320, 1050889, 1670494, 456326, 2141725, 2..."
SESS_SEQ,"[40, 5, 65, 87, 92, 2, 16, 13, 5, 47, 156, 491...","[7, 88, 181, 99, 307, 114, 236, 271, 47, 194, ...","[9, 85, 429, 4, 68, 122, 12, 94, 208, 227, 5, ...","[2, 1, 81, 42, 50, 59, 4, 600, 128, 30, 49, 52...","[21, 15, 86, 15, 17, 165, 366, 397, 362, 517, ..."
TOT_PAG_VIEW_CT,"[41.0, 13.0, 41.0, 36.0, 24.0, 4.0, 159.0, 33....","[207.0, 142.0, 142.0, 285.0, 333.0, 52.0, 267....","[82.0, 154.0, 174.0, 37.0, 185.0, 56.0, 244.0,...","[115.0, 63.0, 221.0, 422.0, 208.0, 250.0, 207....","[105.0, 160.0, 103.0, 160.0, 72.0, 77.0, 53.0,..."
TOT_SESS_HR_V,"[632, 298, 444, 340, 317, 263, 786, 312, 666, ...","[4143, 3352, 3301, 3612, 4103, 4039, 4851, 376...","[1890, 3099, 2019, 1904, 2087, 2543, 3119, 199...","[8172, 6077, 5717, 6399, 5722, 6321, 6698, 639...","[1604, 1035, 976, 1038, 1104, 861, 1022, 860, ..."
DVC_CTG_NM,"[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
ACTION,"[1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, ...","[1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, ...","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


## 구매자가 방문한 SESSION과 구매목록을 정리해서 알기위해 구현한 부분

In [24]:
product = prod[['CLNT_ID','SESS_ID','PD_C']]
product['CLUSTER'] = cluster
print(product)
# cluster 없는 데이터 제거 
product[product['CLUSTER'].isnull()].head()
product = product.dropna()
print(product.tail())

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


         CLNT_ID   SESS_ID    PD_C  CLUSTER
0        4139680   7605037  642112      3.0
1        4140076  10189797  570603      3.0
2        4142395   6158159  179538      0.0
3        4144914   7935714  554336      3.0
4        4144917   6406509  190306      0.0
...          ...       ...     ...      ...
5024901  4132078  10042817  527736      NaN
5024902  4133768   7272433  284643      NaN
5024903  4133768   7946785  284643      NaN
5024904  4134952   5355187  840639      NaN
5024905  4136808   8930953  442584      NaN

[5024906 rows x 4 columns]
         CLNT_ID   SESS_ID    PD_C  CLUSTER
2698691  5726893  10703446  405458      0.0
2698692  5736441   1530644  837751      3.0
2698693  5737065   4105677  707145      1.0
2698694  5741277   4033875  699794      3.0
2698695  5768908   8517578  519123      0.0


In [25]:
data3 = product.groupby('CLNT_ID').agg(lambda x: x.tolist())
data3.head()

,SESS_ID,PD_C,CLUSTER
CLNT_ID,,,
25,"[5317297, 6059256, 8113243, 9506206]","[598634, 715072, 715121, 715121]","[0.0, 0.0, 0.0, 0.0]"
32,"[759569, 759569, 1136737, 1136737]","[31982, 31974, 48881, 48881]","[1.0, 0.0, 0.0, 4.0]"
33,"[10548225, 8694850, 10300429]","[605825, 613253, 806941]","[3.0, 0.0, 1.0]"
56,"[2108568, 2108568]","[425529, 425529]","[0.0, 3.0]"
58,[4328876],[146435],[1.0]


In [26]:
data3.shape
cluster_cod = data3['CLUSTER']
cluster_cod.head()

CLNT_ID
25    [0.0, 0.0, 0.0, 0.0]
32    [1.0, 0.0, 0.0, 4.0]
33         [3.0, 0.0, 1.0]
56              [0.0, 3.0]
58                   [1.0]
Name: CLUSTER, dtype: object

In [27]:
te = TransactionEncoder()
te_ary = te.fit(cluster_cod).transform(cluster_cod)
result = pd.DataFrame(te_ary, columns=te.columns_)
result[:10]
#apriori(, min_support=0.1, use_colnames=True,verbose=1)

,0.0,1.0,2.0,3.0,4.0
0,True,False,False,False,False
1,True,True,False,False,True
2,True,True,False,True,False
3,True,False,False,True,False
4,False,True,False,False,False
5,True,True,False,True,True
6,True,False,False,True,False
7,True,False,False,True,False
8,True,False,False,True,False
9,True,False,False,False,False


In [28]:
frequent_itemsets =  apriori(result, min_support=0.1,  use_colnames=True,verbose=1)
frequent_itemsets

Processing 12 combinations | Sampling itemset size 3


,support,itemsets
0,0.749655,(0.0)
1,0.370389,(1.0)
2,0.547859,(3.0)
3,0.167144,(4.0)
4,0.265335,"(0.0, 1.0)"
5,0.378696,"(0.0, 3.0)"
6,0.125543,"(0.0, 4.0)"
7,0.209824,"(1.0, 3.0)"
8,0.103178,"(3.0, 4.0)"
9,0.165592,"(0.0, 1.0, 3.0)"


In [29]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.749655,(0.0),1
1,0.370389,(1.0),1
2,0.547859,(3.0),1
3,0.167144,(4.0),1
4,0.265335,"(0.0, 1.0)",2
5,0.378696,"(0.0, 3.0)",2
6,0.125543,"(0.0, 4.0)",2
7,0.209824,"(1.0, 3.0)",2
8,0.103178,"(3.0, 4.0)",2
9,0.165592,"(0.0, 1.0, 3.0)",3


### 알 수 있는 점
-cluster0 이 가장 많은 데이터를 포함하고 있으며 cluster2가 가장 적은 데이터를 포함하고 있다. 
-가장 많은 데이터를 갖지만 페이지 방문시간은 작다..! 적은시간을 들여 에 구매하는 사람이 많다는 것을 알 수 있다
또한, 많은 데이터, 적은 action 평균을 갖는 다는 것은 충동구매(buy만)가 많다는 것!

=> 많은 사람들이 짧은 시간에 충동구매를 한다.!

- cluster2가 가장 적은 데이터를 갖지만 action의 평균이 가장 높은데 이는 검색해서 구매하는, 계획 구매가 많다는 것을 알 수 있다
적은 수의 사람이 계획구매를 한다. 

,support,itemsets


In [1]:
#itemset = apriori(item_set, use_colnames=True)
#itemset

In [2]:
#frequent_itemsets = apriori(item_set, min_support=0.1, use_colnames=True)

In [1]:
#association_rules(frequent_itemsets, metric="lift", min_threshold=1)
import pandas as pd

In [65]:
data = df.iloc[:,3:].groupby('CLUSTER').agg(['size','mean','sum'])
data[:5]

TOT_PAG_VIEW_CT                         DVC_CTG_NM                     \
                   size        mean         sum       size      mean      sum   
CLUSTER                                                                         
0               1226491   37.744698  46293532.0    1226491  0.871851  1069318   
1                230012  205.051649  47164340.0     230012  0.816897   187896   
2                459397  131.267126  60303724.0     459397  0.841055   386378   
3                 62732  281.248597  17643287.0      62732  0.728958    45729   
4                720064   82.783961  59609750.0     720064  0.849958   612024   

          ACTION                    
            size      mean     sum  
CLUSTER                             
0        1226491  0.342672  420284  
1         230012  0.785216  180609  
2         459397  0.675499  310322  
3          62732  0.850332   53343  
4         720064  0.557888  401715

In [31]:
df[:5]

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,1,1,3
1,5873599,6616320,21,105.0,1604,1,1,3
2,5873599,5886172,40,41.0,632,1,1,0
3,5873884,1050889,15,160.0,1035,1,1,3
4,5874461,10298270,5,13.0,298,1,0,0


In [68]:
category = df.groupby(['CLUSTER','DVC_CTG_NM']).agg({'DVC_CTG_NM':'size'})
category

DVC_CTG_NM
CLUSTER DVC_CTG_NM            
0       0               161811
        1              1060042
        2                 4638
1       0                43435
        1               185258
        2                 1319
2       0                75352
        1               381712
        2                 2333
3       0                17387
        1                44961
        2                  384
4       0               111364
        1               605376
        2                 3324

In [69]:
category['proportion'] = df.div(df.n.sum(level=0),level=0)
print(cateogory)

AttributeError: 'DataFrame' object has no attribute 'n'

# Itemset For Each Pattern

In [2]:
import pandas as pd
data = pd.read_csv("Clustered_Session.csv")

In [3]:
data[:5]

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,mobile,"buy,search",4
1,5873599,6616320,21,105.0,1604,mobile,"buy,search",4
2,5873599,5886172,40,41.0,632,mobile,"buy,search",0
3,5873884,1050889,15,160.0,1035,mobile,"buy,search",3
4,5874461,10298270,5,13.0,298,mobile,buy,0


In [6]:
category = data.groupby(['CLUSTER','DVC_CTG_NM']).agg({'DVC_CTG_NM':'size'})
category = category.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
category

DVC_CTG_NM
CLUSTER DVC_CTG_NM            
0       desktop      12.971141
        mobile       86.660721
        tablet        0.368138
1       desktop      18.290466
        mobile       81.144108
        tablet        0.565426
2       desktop      26.423231
        mobile       72.987933
        tablet        0.588836
3       desktop      15.166219
        mobile       84.383024
        tablet        0.450757
4       desktop      16.035851
        mobile       83.468456
        tablet        0.495693

In [7]:
category2 = data.groupby(['CLUSTER','ACTION']).agg({'ACTION':'size'})
category2 = category2.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
category2

ACTION
CLUSTER ACTION               
0       buy         67.984256
        buy,search  32.015744
1       buy         23.305983
        buy,search  76.694017
2       buy         15.409348
        buy,search  84.590652
3       buy         47.386090
        buy,search  52.613910
4       buy         35.573682
        buy,search  64.426318

### 여러 클러스터링 사용해보기 
#### KMEANS

In [ ]:
#cols = ['SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']
#for num in range(3,7):
#print("For n_clusters = {}".format(num))
#clusterer = MiniBatchKMeans(n_clusters=5)
#preds = clusterer.fit_predict(df[cols])
#print(clusterer.score(df[cols]))

In [22]:
from sklearn.cluster import KMeans
import matplotlib.pyplot  as plt
import seaborn as sns

In [23]:
cols = ['SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']
#for num in range(3,7):
#print("For n_clusters = {}".format(num))
clusterer = KMeans(n_clusters=5)
preds = clusterer.fit_predict(df[cols])
print(clusterer.score(df[cols]))

-744884914000.578


In [24]:
data['CLUSTER'] = preds
data.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,mobile,"buy,search",3
1,5873599,6616320,21,105.0,1604,mobile,"buy,search",3
2,5873599,5886172,40,41.0,632,mobile,"buy,search",0
3,5873884,1050889,15,160.0,1035,mobile,"buy,search",3
4,5874461,10298270,5,13.0,298,mobile,buy,0


In [26]:
data2 = data.iloc[:,3:].groupby('CLUSTER').agg(['size','mean','sum'])
data2[:5]

TOT_PAG_VIEW_CT                         TOT_SESS_HR_V               \
                   size        mean         sum          size         mean   
CLUSTER                                                                      
0                400891  149.435547  59907366.0        400891  2800.438930   
1               1354833   40.448255  54800630.0       1354833   447.874627   
2                 36987  292.905399  10833692.0         36987  8221.867683   
3                737133   91.147697  67187975.0        737133  1468.354336   
4                168852  226.736846  38284970.0        168852  4746.481694   

                    DVC_CTG_NM                      ACTION                    
                sum       size      mean      sum     size      mean     sum  
CLUSTER                                                                       
0        1122670763     400891  0.835244   334842   400891  0.709602  284473  
1         606795324    1354833  0.870104  1178845  1354833  0.357922  484924  
2         304102220      36987  0.694325    25681    36987  0.857004   31698  
3        1082372437     737133  0.849105   625903   737133  0.581618  428730  
4         801452927     168852  0.805877   136074   168852  0.808092  136448

In [31]:
df = df[['CLNT_ID', 'SESS_ID','SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','DVC_CTG_NM','ACTION']]
df.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,mobile,"buy,search"
1,5873599,6616320,21,105.0,1604,mobile,"buy,search"
2,5873599,5886172,40,41.0,632,mobile,"buy,search"
3,5873884,1050889,15,160.0,1035,mobile,"buy,search"
4,5874461,10298270,5,13.0,298,mobile,buy


In [26]:
df['CLUSTER']=preds
df.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,1,1,3
1,5873599,6616320,21,105.0,1604,1,1,3
2,5873599,5886172,40,41.0,632,1,1,0
3,5873884,1050889,15,160.0,1035,1,1,3
4,5874461,10298270,5,13.0,298,1,0,0


In [27]:
category2 = df.groupby(['CLUSTER','ACTION']).agg({'ACTION':'size'})
category2 = category2.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
category2

ACTION
CLUSTER ACTION           
0       0       64.040327
        1       35.959673
1       0       28.720210
        1       71.279790
2       0       18.976188
        1       81.023812
3       0       41.552870
        1       58.447130
4       0       14.305670
        1       85.694330

In [34]:
category = df.groupby(['CLUSTER','DVC_CTG_NM']).agg({'DVC_CTG_NM':'size'})
category = category.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
category

DVC_CTG_NM
CLUSTER DVC_CTG_NM            
0       desktop      17.004622
        mobile       82.466306
        tablet        0.529071
1       desktop      13.375597
        mobile       86.238452
        tablet        0.385952
2       desktop      31.173115
        mobile       68.221267
        tablet        0.605618
3       desktop      15.556487
        mobile       83.976569
        tablet        0.466944
4       desktop      19.993841
        mobile       79.424585
        tablet        0.581574

In [28]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [29]:
data2 = df.groupby('CLUSTER').agg(lambda x: x.tolist())
data2.transpose()

CLUSTER,0,1,2,3,4
CLNT_ID,"[5873599, 5874461, 5874701, 5874829, 5874829, ...","[5874829, 5878622, 5878892, 5914916, 5917727, ...","[5874900, 6075137, 6070142, 6072805, 6072961, ...","[5873599, 5873599, 5873884, 5874829, 5876333, ...","[5878512, 5880431, 5879521, 5942318, 5946440, ..."
SESS_ID,"[5886172, 10298270, 608577, 1666766, 1559438, ...","[1694242, 352440, 1554809, 2188919, 612280, 20...","[10229983, 3237950, 1111137, 841910, 10312575,...","[8641867, 6616320, 1050889, 1670494, 456326, 2...","[658549, 8358105, 10801689, 1262876, 318952, 9..."
SESS_SEQ,"[40, 5, 65, 87, 92, 2, 16, 13, 5, 47, 156, 165...","[85, 88, 122, 12, 181, 99, 208, 227, 159, 217,...","[7, 307, 114, 1, 236, 81, 182, 28, 42, 50, 59,...","[9, 21, 15, 86, 15, 17, 366, 429, 517, 368, 33...","[2, 4, 128, 49, 52, 1, 3, 3, 243, 90, 109, 296..."
TOT_PAG_VIEW_CT,"[41.0, 13.0, 41.0, 36.0, 24.0, 4.0, 159.0, 33....","[154.0, 142.0, 56.0, 244.0, 142.0, 285.0, 101....","[207.0, 333.0, 52.0, 63.0, 267.0, 221.0, 276.0...","[82.0, 105.0, 160.0, 103.0, 160.0, 72.0, 53.0,...","[115.0, 207.0, 361.0, 467.0, 250.0, 277.0, 265..."
TOT_SESS_HR_V,"[632, 298, 444, 340, 317, 263, 786, 312, 666, ...","[3099, 3352, 2543, 3119, 3301, 3612, 2546, 238...","[4143, 4103, 4039, 6077, 4851, 5717, 4830, 425...","[1890, 1604, 1035, 976, 1038, 1104, 1022, 2019...","[8172, 6698, 6831, 7241, 7541, 7081, 10502, 75..."
DVC_CTG_NM,"[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
ACTION,"[1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, ...","[1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, ..."


In [30]:
product = prod[['CLNT_ID','SESS_ID','PD_C']]
product['CLUSTER'] = cluster
print(product)
# cluster 없는 데이터 제거 
product[product['CLUSTER'].isnull()].head()
product = product.dropna()
print(product.tail())

NameError: name 'cluster' is not defined

In [31]:
data3 = product.groupby('CLNT_ID').agg(lambda x: x.tolist())
data3.head()

,SESS_ID,PD_C
CLNT_ID,,
8,"[6964877, 7017688]","[206218, 206218]"
14,"[1156991, 1156991]","[645262, 369141]"
25,"[5317297, 6059256, 8113243, 9506206]","[598634, 715072, 715121, 715121]"
32,"[759569, 759569, 1136737, 1136737, 1136737]","[31982, 31974, 48881, 48881, 38697]"
33,"[10548225, 8694850, 10300429, 9521159, 8694850]","[605825, 613253, 806941, 607160, 607160]"


In [44]:
data3.shape
cluster_cod = data3['CLUSTER']
cluster_cod.head()

te = TransactionEncoder()
te_ary = te.fit(cluster_cod).transform(cluster_cod)
result = pd.DataFrame(te_ary, columns=te.columns_)
result[:10]

KeyError: 'CLUSTER'

In [3]:
import pandas as pd
data = pd.read_csv('cluster.csv')
data

,Unnamed: 0,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,0,5873599,8641867,9,82.0,1890,1,1,0
1,1,5873599,6616320,21,105.0,1604,1,1,0
2,2,5873599,5886172,40,41.0,632,1,1,0
3,3,5873884,1050889,15,160.0,1035,1,1,0
4,4,5874461,10298270,5,13.0,298,1,0,1
...,...,...,...,...,...,...,...,...,...
2698691,2698691,3500464,8713975,308,71.0,423,1,0,1
2698692,2698692,3501155,4636571,86,44.0,2090,0,1,0
2698693,2698693,3501397,582632,128,164.0,2435,1,0,1
2698694,2698694,3573271,2328407,9,5.0,986,0,0,1


### ACTION 과 tot pag count..

In [19]:
prod = pd.read_csv('01.Pruduct.csv')
sess = pd.read_csv('Clustered_Session.csv')


C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
sess = sess.drop('CLUSTER', axis=1)
sess

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,mobile,"buy,search"
1,5873599,6616320,21,105.0,1604,mobile,"buy,search"
2,5873599,5886172,40,41.0,632,mobile,"buy,search"
3,5873884,1050889,15,160.0,1035,mobile,"buy,search"
4,5874461,10298270,5,13.0,298,mobile,buy
...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,mobile,buy
2698692,3501155,4636571,86,44.0,2090,desktop,"buy,search"
2698693,3501397,582632,128,164.0,2435,mobile,buy
2698694,3573271,2328407,9,5.0,986,desktop,buy


In [33]:
from sklearn.preprocessing import LabelEncoder

encoded = sess
encode_ACT = LabelEncoder()
encode_ACT.fit(sess['ACTION'])

encoded['ACTION'] = encode_ACT.transform(sess['ACTION'])

encoded.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,mobile,1,3
1,5873599,6616320,21,105.0,1604,mobile,1,3
2,5873599,5886172,40,41.0,632,mobile,1,1
3,5873884,1050889,15,160.0,1035,mobile,1,0
4,5874461,10298270,5,13.0,298,mobile,0,1


In [34]:
from sklearn.cluster import MiniBatchKMeans, KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

cols =['TOT_PAG_VIEW_CT','ACTION']
clusterer = MiniBatchKMeans(n_clusters=5)
preds = clusterer.fit_predict(encoded[cols])
print(clusterer.score(encoded[cols]))

-1159405125.9850094


In [35]:
data = encoded
data['CLUSTER'] = preds
data.shape

(2698696, 8)

In [36]:
data

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,mobile,1,2
1,5873599,6616320,21,105.0,1604,mobile,1,2
2,5873599,5886172,40,41.0,632,mobile,1,0
3,5873884,1050889,15,160.0,1035,mobile,1,4
4,5874461,10298270,5,13.0,298,mobile,0,0
...,...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,mobile,0,2
2698692,3501155,4636571,86,44.0,2090,desktop,1,0
2698693,3501397,582632,128,164.0,2435,mobile,0,4
2698694,3573271,2328407,9,5.0,986,desktop,0,0


In [31]:
data2 = data.groupby(['CLUSTER']).agg(['size','mean'])
data2

CLNT_ID                SESS_ID               SESS_SEQ              \
            size          mean     size          mean     size        mean   
CLUSTER                                                                      
0         393542  3.366749e+06   393542  5.744689e+06   393542  226.960119   
1        1246356  3.280352e+06  1246356  5.844265e+06  1246356  134.921132   
2          89226  3.409254e+06    89226  5.727296e+06    89226  304.315256   
3         772531  3.328725e+06   772531  5.767383e+06   772531  190.292202   
4         197041  3.393529e+06   197041  5.721997e+06   197041  258.448257   

        TOT_PAG_VIEW_CT             TOT_SESS_HR_V                ACTION  \
                   size        mean          size         mean     size   
CLUSTER                                                                   
0                393542  140.841295        393542  2296.829751   393542   
1               1246356   27.560217       1246356   666.722119  1246356   
2                 89226  408.444837         89226  4756.295396    89226   
3                772531   74.450866        772531  1416.901796   772531   
4                197041  239.941560        197041  3367.501231   197041   

                   
             mean  
CLUSTER            
0        0.751991  
1        0.303706  
2        0.933764  
3        0.566517  
4        0.867023

buysearch 가 1이고 buy가 0이므로 action의 mean이 1에 가까울수록 검색하고구매한 비율이 많음을 알 수 있다. cluster2의 buysearch비율이 많고, 페이지 조회수는 가장낮은데, 신기하게?예상과 다르게 tot_sess_hr_v 즉, 세션에 머무른 시간을 보면 확연히 size(갯수)는 작은데 mean이 높다. 그럼 머무르는 시간이 크다고 볼 수 있나? (=> 검색한 후 구매해서 총 페이지 조회수는 적지만 한 세션당 머무르는 시간은 확실히 길다.-> 상품을 꼼꼼히 고려해서 산다는 것을 알수있나?)
그 다음으로 action에서 buysearch의 비율이 높은 cluster4번도 페이지 조회수는 2번째로 적은데 세션에 머무르는 시간은 두번째로 길다.
가장 검색없이 구매하는 비율이 높다고 볼 수 있는 cluster1 은 가장 적은 페이지 조회, 세션당 머무르는 시간을 보인다. 그런데 총 클러스터가 포함하는 갯수는 가장 많은 것을 보니. 많은 비율의 사람들이 이런식으로 구매하는 것 같다. 

=> 검색하고 구매하면, 한 세션당 머무른 시간이 길다 즉, 제품에 대해 천천히 탐색하는 시간을 갖는 것 같다.  => 고민 많이 하고 구매
=> 검색없이 구매하면, 페이지 조회수 평균도 가장 짧고, 세션당 머무는 시간도 가장 짧다. -> 바로 구매해버리는 것 같다. => 고민없이 구매

검색구매가 높은 순서대로
2 > 4 > 0 > 3 >> 1 

In [39]:
data3 = data.groupby(['CLUSTER','DVC_CTG_NM']).agg({'DVC_CTG_NM':'size'})
data3

DVC_CTG_NM
CLUSTER DVC_CTG_NM            
0       desktop         314453
        mobile         1028777
        tablet            5500
1       desktop           1362
        mobile           73480
        tablet             446
2       desktop          66459
        mobile          668569
        tablet            3268
3       desktop           5973
        mobile          163272
        tablet             970
4       desktop          21102
        mobile          343251
        tablet            1814

위의 표는 세션조회수와 action을 클러스터 했을 때의 클러스터를 기준으로 기기의 유형에 따른 분포정도를 보기위해 출력함
전체적으로 각 클러스터마다 mobile > desktop > tablet 순서로 구매를 많이 한다.

cluster2가 검색구매의 비율이 가장 높은 클러스터이다 (0.9의 평균을 가짐)


In [40]:
ratio = data.groupby(['CLUSTER','DVC_CTG_NM']).agg({'DVC_CTG_NM':'size'})
ratio = ratio.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
ratio

DVC_CTG_NM
CLUSTER DVC_CTG_NM            
0       desktop      23.314748
        mobile       76.277461
        tablet        0.407791
1       desktop       1.809053
        mobile       97.598555
        tablet        0.592392
2       desktop       9.001674
        mobile       90.555685
        tablet        0.442641
3       desktop       3.509091
        mobile       95.921041
        tablet        0.569868
4       desktop       5.762944
        mobile       93.741653
        tablet        0.495402

비율로 보면 위와 같다.
검색구매가 높은 순서대로 2 > 4 > 0 > 3 >> 1 이다.

전체적으로 mobile > destop > tablet 순서로 높다. 

cluster0은 세번째로 검색해서 구매하는 비율이 높은데..그리고 모든 attribute(TOT_PAG_CT, HOUR 등등) 에서 세번째인데 (중간)  
왜 desktop의 비율이 다른 곳 보다 더 높을까 ? -> 예상과는 다른 양상을 보인다. 분석해보면 재밌는 결과가 나오지 않을까?

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoded = sess
encode_ACT = LabelEncoder()
encode_ACT.fit(sess['ACTION'])

encoded['ACTION'] = encode_ACT.transform(sess['ACTION'])

encoded.head()

## ACTION 과 DVC_CTG_NM 클러스터링

In [56]:
sess

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,mobile,"buy,search"
1,5873599,6616320,21,105.0,1604,mobile,"buy,search"
2,5873599,5886172,40,41.0,632,mobile,"buy,search"
3,5873884,1050889,15,160.0,1035,mobile,"buy,search"
4,5874461,10298270,5,13.0,298,mobile,buy
...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,mobile,buy
2698692,3501155,4636571,86,44.0,2090,desktop,"buy,search"
2698693,3501397,582632,128,164.0,2435,mobile,buy
2698694,3573271,2328407,9,5.0,986,desktop,buy


In [57]:
encod = sess
encode_ACT = LabelEncoder()
encode_ACT.fit(sess['DVC_CTG_NM'])

encod['DVC_CTG_NM'] = encode_ACT.transform(sess['DVC_CTG_NM'])

encode_ACT = LabelEncoder()
encode_ACT.fit(sess['ACTION'])

encod['ACTION'] = encode_ACT.transform(sess['ACTION'])


encod.head()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,1,1
1,5873599,6616320,21,105.0,1604,1,1
2,5873599,5886172,40,41.0,632,1,1
3,5873884,1050889,15,160.0,1035,1,1
4,5874461,10298270,5,13.0,298,1,0


In [58]:
encod.tail()

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
2698691,3500464,8713975,308,71.0,423,1,0
2698692,3501155,4636571,86,44.0,2090,0,1
2698693,3501397,582632,128,164.0,2435,1,0
2698694,3573271,2328407,9,5.0,986,0,0
2698695,3573758,8702345,17,18.0,373,0,0


## 클러스터 갯수 4

In [61]:
from sklearn.cluster import MiniBatchKMeans, KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

cols =['DVC_CTG_NM','ACTION']
clusterers = MiniBatchKMeans(n_clusters=4)
predss = clusterers.fit_predict(encod[cols])
print(clusterers.score(encod[cols]))

-11953.915385400556


In [66]:
data4 = encod
data4['CLUSTER'] = predss
data4.shape
data4

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,1,1,1
1,5873599,6616320,21,105.0,1604,1,1,1
2,5873599,5886172,40,41.0,632,1,1,1
3,5873884,1050889,15,160.0,1035,1,1,1
4,5874461,10298270,5,13.0,298,1,0,0
...,...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,1,0,0
2698692,3501155,4636571,86,44.0,2090,0,1,3
2698693,3501397,582632,128,164.0,2435,1,0,0
2698694,3573271,2328407,9,5.0,986,0,0,2


In [64]:
data4 = data4.groupby(['CLUSTER']).agg(['size','mean'])
data4

CLNT_ID                SESS_ID               SESS_SEQ              \
            size          mean     size          mean     size        mean   
CLUSTER                                                                      
0        1131423  3.318057e+06  1131423  5.793352e+06  1131423  181.713162   
1        1157924  3.360330e+06  1157924  5.761764e+06  1157924  228.021445   
2         201000  3.210675e+06   201000  5.981843e+06   201000   33.829736   
3         208349  3.203119e+06   208349  5.807657e+06   208349   29.454094   

        TOT_PAG_VIEW_CT             TOT_SESS_HR_V              DVC_CTG_NM  \
                   size        mean          size         mean       size   
CLUSTER                                                                     
0               1131423   55.649616       1131423   951.462464    1131423   
1               1157924  130.029212       1157924  1839.846716    1157924   
2                201000   27.598632        201000  1191.244358     201000   
3                208349   57.308204        208349  2260.842965     208349   

                    ACTION       
             mean     size mean  
CLUSTER                          
0        1.005178  1131423    0  
1        1.005302  1157924    1  
2        0.000000   201000    0  
3        0.000000   208349    1

In [67]:
ratio = data4.groupby(['CLUSTER','DVC_CTG_NM']).agg({'DVC_CTG_NM':'size'})
ratio = ratio.groupby(level=0).apply(lambda x:100 * x/float(x.sum()))
ratio

DVC_CTG_NM
CLUSTER DVC_CTG_NM            
0       1            99.482157
        2             0.517843
1       1            99.469827
        2             0.530173
2       0           100.000000
3       0           100.000000

In [43]:
from sklearn.cluster import MiniBatchKMeans, KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
%matplotlib inline

cols =['DVC_CTG_NM','ACTION']
clusterer = MiniBatchKMeans(n_clusters=5)
preds = clusterer.fit_predict(encod[cols])
print(clusterer.score(encod[cols]))

-6109.4419687140435


In [44]:
data = encoded
data['CLUSTER'] = preds
data.shape

(2698696, 8)

In [45]:
data

,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION,CLUSTER
0,5873599,8641867,9,82.0,1890,1,1,0
1,5873599,6616320,21,105.0,1604,1,1,0
2,5873599,5886172,40,41.0,632,1,1,0
3,5873884,1050889,15,160.0,1035,1,1,0
4,5874461,10298270,5,13.0,298,1,0,1
...,...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,1,0,1
2698692,3501155,4636571,86,44.0,2090,0,1,3
2698693,3501397,582632,128,164.0,2435,1,0,1
2698694,3573271,2328407,9,5.0,986,0,0,2


In [46]:
data2 = data.groupby(['CLUSTER']).agg(['size','mean'])
data2

CLNT_ID                SESS_ID               SESS_SEQ              \
            size          mean     size          mean     size        mean   
CLUSTER                                                                      
0        1157924  3.360330e+06  1157924  5.761764e+06  1157924  228.021445   
1        1125564  3.318559e+06  1125564  5.793004e+06  1125564  181.995495   
2         201000  3.210675e+06   201000  5.981843e+06   201000   33.829736   
3         208349  3.203119e+06   208349  5.807657e+06   208349   29.454094   
4           5859  3.221739e+06     5859  5.860185e+06     5859  127.474654   

        TOT_PAG_VIEW_CT             TOT_SESS_HR_V              DVC_CTG_NM  \
                   size        mean          size         mean       size   
CLUSTER                                                                     
0               1157924  130.029212       1157924  1839.846716    1157924   
1               1125564   55.634534       1125564   950.313107    1125564   
2                201000   27.598632        201000  1191.244358     201000   
3                208349   57.308204        208349  2260.842965     208349   
4                  5859   58.547022          5859  1172.263697       5859   

                    ACTION       
             mean     size mean  
CLUSTER                          
0        1.005302  1157924    1  
1        1.000000  1125564    0  
2        0.000000   201000    0  
3        0.000000   208349    1  
4        2.000000     5859    0

action과 dvc_ctg_nm 을 클러스터링해서 보인 결과이다.
desktop = 0, mobile = 1, tablet=2
(단, 딱 떨어지는 평균의 경우 그 데이터가 모두 같은류의 데이터의 모임이라고 판단 ex) DVC_CTG_NM 의 평균이 1.0000 이면 모든 데이터가 mobile을 사용한 데이터!)

보면, cluster1은 전부 mobile을 사용하였고, 검색없이 구매한 데이터들의 모임이다.
cluster2 은 전부 desktop을 사용한 것과 검색없이 구매한 데이터들이 모였다.  (둘다 평균이 0이므로!)
cluster3는 전부 desktop을 사용하였고 전부 검색없이 구매한 데이터의 모임
cluster4는 평균이 2인것으로보아 tablet을 사용한구매자모두가 검색없이 구매한 데이터들이 모인것 같다.
단, cluster0은 device의 평균이 1.005.. 인 것을 보아 기기 3종류가 섞였지만 mobile의 비율이 높다는 것을 추측할 수 있다. 이때의 action은 1로 검색후 구매한 데이터들이 모였다.

결론..
tablet이나 mobile을 사용해 구매하는 구매자들은 검색없이 구매할 확률이 높고, 
destkop을 사용해 구매할 경우 검색후 구매, 검색없이 구매의 데이터 size도 거의 비슷하므로 desktop구매자들은 검색구매의 확률의 반반이라고 판단하였다.



## 더 나아가 각 기기별 구매자들이 어떤 제품군 위주로 샀는지 분석해보고자 하는데.. 지금 시간관계상 시간 더 필요함 ㅠㅠ
## 또한, 어떤 제품근이 시간이 오래걸리는지? 그런것도 보면 좋을 것 같다.
위의 분석에는 클러스터 없이 그냥 데이터를 가지고 비교분석해볼 수 있을 것 같다.

In [49]:
prod = pd.read_csv('01.Pruduct.csv')
sess = pd.read_csv('Clustered_Session.csv')
sess = sess.drop('CLUSTER', axis=1)
sess

C:\Users\User\anaconda3.7\envs\chaelin_virtenv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CLNT_ID,SESS_ID,SESS_SEQ,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ACTION
0,5873599,8641867,9,82.0,1890,mobile,"buy,search"
1,5873599,6616320,21,105.0,1604,mobile,"buy,search"
2,5873599,5886172,40,41.0,632,mobile,"buy,search"
3,5873884,1050889,15,160.0,1035,mobile,"buy,search"
4,5874461,10298270,5,13.0,298,mobile,buy
...,...,...,...,...,...,...,...
2698691,3500464,8713975,308,71.0,423,mobile,buy
2698692,3501155,4636571,86,44.0,2090,desktop,"buy,search"
2698693,3501397,582632,128,164.0,2435,mobile,buy
2698694,3573271,2328407,9,5.0,986,desktop,buy


In [54]:
device = sess.groupby('DVC_CTG_NM').agg(['size','mean'])
device = device.drop(['CLNT_ID','SESS_ID','SESS_SEQ'], axis=1)
device

TOT_PAG_VIEW_CT            TOT_SESS_HR_V             
                      size       mean          size         mean
DVC_CTG_NM                                                      
desktop             409349  42.720104        409349  1735.644858
mobile             2277349  93.256100       2277349  1399.341685
tablet               11998  95.892232         11998  1676.929155

In [55]:
act = sess.groupby('ACTION').agg(['size','mean'])
act = act.drop(['CLNT_ID','SESS_ID','SESS_SEQ'], axis=1)
act

TOT_PAG_VIEW_CT             TOT_SESS_HR_V             
                      size        mean          size         mean
ACTION                                                           
buy                1332423   51.418041       1332423   987.634281
buy,search         1366273  118.939664       1366273  1904.046292